## ADCIRC TriMesh data with Datashader

[Datashader](http://datashader.org) support for irregular triangular meshes allows large ADCIRC datasets to be rendered onscreen efficiently. This notebook shows an example of rendering the depth or M2 Amplitude from the [EC2015 tidal database](https://doi.org/10.3390/jmse4040072).  Make possible by the [EarthSim Project](https://pyviz.github.io/EarthSim/).

After the data loads (after ~20-30 seconds), click one of the zoom tools to see performant rendering on the fly....

In [ ]:
import datashader as ds
import pandas as pd
import numpy as np
import holoviews as hv
from holoviews.operation.datashader import datashade, rasterize
import geoviews as gv
import palettable
import ipywidgets as ipyw
from gridgeo.ugrid import ugrid
import netCDF4
hv.extension("bokeh")

In [ ]:
from IPython.display import Javascript, display
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

In [ ]:
# EC2015 data in netCDF form, accessed via OPeNDAP. 
# Here we use a UGRID-ized version of http://tds.renci.org:8080/thredds/dodsC/DataLayers/Tides/ec2015_tidaldb/f53.nc.html
url='http://gamone.whoi.edu/thredds/dodsC/usgs/vault0/models/tides/ec2015/f53.ncml'
nc = netCDF4.Dataset(url)
# Use gridgeo to get mesh from UGRID compliant dataset
u = ugrid(nc)
tris = pd.DataFrame(u['faces'].astype('int'), columns=['v0','v1','v2'])

In [ ]:
vars = ['Depth','M2 Elevation Amplitude',
        'S2 Elevation Amplitude','N2 Elevation Amplitude',
        'O1 Elevation Amplitude','K1 Elevation Amplitude']
var = ipyw.Dropdown(options=vars, value=vars[0])
display(var)

In [ ]:
button = ipyw.Button(button_style='info',description="Execute")
button.on_click(run_all)
display(button)

In [ ]:
if var.value == 'M2 Elevation Amplitude':
    z = nc['Amp'][0,:]   
elif var.value == 'S2 Elevation Amplitude':
    z = nc['Amp'][1,:]   
elif var.value == 'N2 Elevation Amplitude':
    z = nc['Amp'][2,:]        
elif var.value == 'O1 Elevation Amplitude':
    z = nc['Amp'][3,:]    
elif var.value == 'K1 Elevation Amplitude':
    z = nc['Amp'][4,:]    
else:
    z = -nc['depth'][:]

In [ ]:
v = np.vstack((u['nodes']['x'], u['nodes']['y'], z)).T
verts = pd.DataFrame(v, columns=['x','y','z'])

In [ ]:
%opts Image [colorbar=True clipping_colors={'NaN': (0, 0, 0, 0)}] (cmap=palettable.cubehelix.perceptual_rainbow_16.mpl_colormap)
%opts WMTS [width=700 height=400 title_format='{} (m)'.format(var.value)]
tiles = gv.WMTS('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg')
points = gv.operation.project_points(gv.Points(verts, vdims=['z']))
tiles * rasterize(hv.TriMesh((tris, points)), aggregator=ds.mean('z'),  precompute=True ) 